# BT4012 Project Group 7

## 1. Automated Feature Engineering
Now that we have cleaned and preprocessed our data, we can engineer features relevant to predicting fraudulent providers. Instead of generating it manually, we will make use of a technique known as automated feature engineering to help us generate more features. In particular, we will utilise deep feature synthesis from the featuretools library to help us with this process. Deep feature synthesis applies mathematical operations on existing features and stacks them together to create new features. Subsequently, we can test and see the effectiveness of these new features in helping us to predict medicare fraud.

In [2]:
# Please uncomment and run this line if you don't have featuretools installed
# !pip install featuretools
import pandas as pd
import featuretools as ft

### 1.1 Import Preprocessed Datasets 

**Note: The X_test.csv does not contain labels, while the X_train1.csv and X_train2.csv contains labels.**

Since we are going to be creating supervised and unsupervised models, it is important to note that we can only use the X_train1.csv and X_train2.csv to train and test our supervised models.

Hence to avoid confusion on the nature of the dataset when importing, we will import X_train1.csv, X_train2.csv and y_train.csv as X_labeled1, X_labeled2 and y_labels respectively. In addition, we will import X_test as X_test_unlabeled. 

Further down, x_train, x_test, y_train and y_test **<u>will only be obtained from the labeled data</u>** when we have to create our supervised models.

In [3]:
# Load preprocessed datasets
X_train1 = pd.read_csv('X_train1.csv')
X_train2 = pd.read_csv('X_train2.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')
X_labeled_full = pd.concat([X_train1, X_train2]) # Combine into 1 training set

# As labels were provided for fraudulent providers only, we will retain the providers in our y_labels only.
y_providers = y_train[['Provider', 'PotentialFraud']].drop_duplicates(ignore_index=True)

# Next we drop the labels in X_labeled_full
X_train = X_labeled_full[X_labeled_full.columns.difference(['BeneID'])]

display(X_train.head(2))
display(y_providers.head(2))

,AdmitForDays,Age,AttendingPhysician_ce,ChronicCond_Alzheimer,ChronicCond_Cancer,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_Heartfailure,ChronicCond_IschemicHeart,ChronicCond_KidneyDisease,...,OPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OperatingPhysician_ce,OtherPhysician_ce,Provider,Race,RenalDiseaseIndicator,State,isDead,is_inpatient
0,7.0,67.0,1.0,1,0,1,1,0,1,1,...,70,60,0.0,0.0,PRV55912,1,0,39,0.0,1
1,3.0,67.0,3.0,1,0,1,1,0,1,1,...,70,60,3.0,0.0,PRV55907,1,0,39,0.0,1


,Provider,PotentialFraud
0,PRV55912,1
1,PRV55907,0


AutoEncoders Model

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

In [ ]:
# Step 1: Preprocess the data
features = df.drop(['Provider', 'Fraud'], axis=1).values

# Standardize the features
scaler = StandardScaler()
features = scaler.fit_transform(features)

In [10]:
class Autoencoder(Model):
    def __init__(self, latent_dim, shape):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.shape = shape
        self.encoder = tf.keras.Sequential([
            layers.Flatten(),
            layers.Dense(latent_dim, activation='relu'),
        ])
        self.decoder = tf.keras.Sequential([
            layers.Dense(tf.math.reduce_prod(shape), activation='sigmoid'),
            layers.Reshape(shape)
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


shape = x_test.shape[1:]
latent_dim = 64
autoencoder = Autoencoder(latent_dim, shape)

NameError: name 'x_test' is not defined

### 6.2 Train-Val Split

As deep feature synthesis carries out aggregation operations on our data, we will have to scale our data to avoid imbalancing issues. However, to avoid data leakage, we will need to prepare our scaler on training data and then use it to fit our test data. 

Thus we will first train-test split our data.

**Note: As we are aggregating over Provider, we will utilise GroupShuffleSplit to split the data by Provider to ensure that there is no train-test contamination.**

In [9]:
from sklearn.model_selection import GroupShuffleSplit

gss = GroupShuffleSplit(n_splits=2, test_size=.2, random_state=4012)
split = gss.split(X_train, groups=X_train.Provider)
train_ix, test_ix = next(split)

X_train = X_full.loc[train_ix]
X_test = X_full.loc[test_ix]

providers_train = X_train.Provider.unique()
providers_test = X_test.Provider.unique()
y_train = y_providers[y_providers.Provider.isin(providers_train)]
y_test = y_providers[y_providers.Provider.isin(providers_test)]

KeyError: '[279106, 279107, 279108, 279109, 279110, 279111, 279112, 279113, 279114, 279115, 279117, 279118, 279119, 279121, 279122, 279123, 279124, 279125, 279127, 279128, 279129, 279130, 279131, 279132, 279133, 279134, 279135, 279136, 279137, 279138, 279139, 279140, 279141, 279142, 279143, 279146, 279148, 279154, 279155, 279156, 279157, 279158, 279159, 279160, 279162, 279163, 279164, 279165, 279166, 279167, 279168, 279169, 279170, 279171, 279172, 279173, 279174, 279175, 279176, 279177, 279178, 279179, 279180, 279181, 279182, 279183, 279184, 279185, 279186, 279187, 279188, 279189, 279190, 279191, 279192, 279193, 279194, 279195, 279196, 279197, 279198, 279199, 279200, 279202, 279203, 279204, 279205, 279207, 279209, 279210, 279211, 279212, 279213, 279214, 279215, 279216, 279217, 279218, 279219, 279221, 279222, 279223, 279224, 279226, 279228, 279229, 279231, 279232, 279233, 279234, 279235, 279236, 279237, 279238, 279239, 279242, 279243, 279244, 279245, 279246, 279247, 279248, 279249, 2792

In [287]:
# View X_train
display(X_train.head())
print(X_train.shape)

,AdmitForDays,Age,AttendingPhysician_ce,ChronicCond_Alzheimer,ChronicCond_Cancer,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_Heartfailure,ChronicCond_IschemicHeart,ChronicCond_KidneyDisease,ChronicCond_ObstrPulmonary,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,ClaimID,ClaimProcessedForDays,ClmAdmitDiagnosisCode_ce,ClmDiagnosisCode_10_ce,ClmDiagnosisCode_1_ce,ClmDiagnosisCode_2_ce,ClmDiagnosisCode_3_ce,ClmDiagnosisCode_4_ce,ClmDiagnosisCode_5_ce,ClmDiagnosisCode_6_ce,ClmDiagnosisCode_7_ce,ClmDiagnosisCode_8_ce,ClmDiagnosisCode_9_ce,ClmProcedureCode_1_ce,ClmProcedureCode_2_ce,ClmProcedureCode_3_ce,ClmProcedureCode_4_ce,ClmProcedureCode_5_ce,County,DeductibleAmtPaid,DiagnosisGroupCode_ce,Gender,IPAnnualDeductibleAmt,IPAnnualReimbursementAmt,InscClaimAmtReimbursed,NoOfMonths_PartACov,NoOfMonths_PartBCov,OPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OperatingPhysician_ce,OtherPhysician_ce,Provider,Race,RenalDiseaseIndicator,State,isDead,is_inpatient
1,3.0,67.0,3.0,1,0,1,1,0,1,1,0,0,1,1,CLM66048,3,57.0,0.0,57.0,5182.0,3461.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,230,1068.0,16,1,3204,36000,5000,12,12,70,60,3.0,0.0,PRV55907,1,0,39,0.0,1
2,4.0,67.0,12.0,1,0,1,1,0,1,1,0,0,1,1,CLM68358,4,1041.0,0.0,170.0,550.0,3416.0,2177.0,836.0,2242.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,230,1068.0,160,1,3204,36000,5000,12,12,70,60,0.0,14.0,PRV56046,1,0,39,0.0,1
3,9.0,96.0,55.0,0,0,1,1,1,0,1,0,0,1,1,CLM38412,9,360.0,0.0,2077.0,1423.0,630.0,2242.0,163.0,2941.0,412.0,5427.0,79788.0,100.0,0.0,0.0,0.0,0.0,360,1068.0,38,2,1068,5000,5000,12,12,320,250,12.0,15.0,PRV52405,2,0,1,0.0,1
6,6.0,70.0,8.0,1,1,0,0,1,1,0,0,1,1,0,CLM32075,6,173.0,0.0,185.0,7534.0,652.0,1290.0,13398.0,79788.0,5182.0,8333.0,4499.0,82.0,0.0,0.0,0.0,0.0,120,1068.0,64,1,1068,8000,8000,12,12,50,860,4.0,0.0,PRV54090,2,0,24,0.0,1
7,5.0,68.0,5.0,0,0,0,1,0,1,0,0,1,0,0,CLM62376,5,7304.0,0.0,327.0,39724.0,4984.0,7534.0,1009.0,15171.0,2197.0,4499.0,15246.0,1152.0,0.0,0.0,0.0,0.0,230,1068.0,79,1,1068,6000,6000,12,12,0,0,19.0,0.0,PRV51148,1,0,38,0.0,1


(447666, 51)


In [288]:
# View y_train
display(y_train.head())
print(y_train.shape)

,Provider,PotentialFraud
1,PRV55907,0
2,PRV56046,0
3,PRV52405,0
6,PRV54090,0
7,PRV51148,1


(4328, 2)


In [289]:
# View X_test
display(X_test.head())
print(X_test.shape)

,AdmitForDays,Age,AttendingPhysician_ce,ChronicCond_Alzheimer,ChronicCond_Cancer,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_Heartfailure,ChronicCond_IschemicHeart,ChronicCond_KidneyDisease,ChronicCond_ObstrPulmonary,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,ClaimID,ClaimProcessedForDays,ClmAdmitDiagnosisCode_ce,ClmDiagnosisCode_10_ce,ClmDiagnosisCode_1_ce,ClmDiagnosisCode_2_ce,ClmDiagnosisCode_3_ce,ClmDiagnosisCode_4_ce,ClmDiagnosisCode_5_ce,ClmDiagnosisCode_6_ce,ClmDiagnosisCode_7_ce,ClmDiagnosisCode_8_ce,ClmDiagnosisCode_9_ce,ClmProcedureCode_1_ce,ClmProcedureCode_2_ce,ClmProcedureCode_3_ce,ClmProcedureCode_4_ce,ClmProcedureCode_5_ce,County,DeductibleAmtPaid,DiagnosisGroupCode_ce,Gender,IPAnnualDeductibleAmt,IPAnnualReimbursementAmt,InscClaimAmtReimbursed,NoOfMonths_PartACov,NoOfMonths_PartBCov,OPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OperatingPhysician_ce,OtherPhysician_ce,Provider,Race,RenalDiseaseIndicator,State,isDead,is_inpatient
0,7.0,67.0,1.0,1,0,1,1,0,1,1,0,0,1,1,CLM46614,7,955.0,0.0,748.0,79788.0,4251.0,630.0,4673.0,6980.0,37270.0,341.0,5266.0,0.0,0.0,0.0,0.0,0.0,230,1068.0,129,1,3204,36000,26000,12,12,70,60,0.0,0.0,PRV55912,1,0,39,0.0,1
4,18.0,72.0,2.0,0,0,1,0,1,1,1,1,0,0,0,CLM63689,18,1937.0,0.0,680.0,9099.0,292.0,5384.0,2765.0,6242.0,2442.0,580.0,831.0,854.0,0.0,0.0,0.0,0.0,780,1068.0,5,2,2136,21260,10000,12,12,100,120,5.0,0.0,PRV56614,1,1,45,0.0,1
5,7.0,70.0,50.0,1,1,0,1,1,1,1,1,0,1,1,CLM70950,7,1915.0,0.0,405.0,768.0,2275.0,10181.0,158.0,203.0,5182.0,39724.0,3085.0,13.0,0.0,0.0,0.0,0.0,270,1068.0,35,2,2136,22000,8000,12,12,840,1400,50.0,50.0,PRV54986,1,0,31,0.0,1
10,9.0,64.0,31.0,1,0,1,1,1,1,1,1,0,1,1,CLM57949,9,7304.0,0.0,200.0,296.0,501.0,1423.0,5266.0,12185.0,805.0,868.0,2574.0,307.0,0.0,0.0,0.0,0.0,760,1068.0,90,2,2136,131140,102000,12,12,80,1650,23.0,0.0,PRV55193,1,0,34,0.0,1
13,6.0,87.0,19.0,0,0,1,1,1,1,1,1,0,1,0,CLM54944,6,3790.0,0.0,3790.0,1143.0,4603.0,3416.0,7938.0,15171.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,150,1068.0,153,2,2136,23000,6000,12,12,370,620,0.0,0.0,PRV52283,1,0,10,0.0,1


(110545, 51)


In [290]:
# View y_test
display(y_test.head())
print(y_test.shape)

,Provider,PotentialFraud
0,PRV55912,1
4,PRV56614,0
5,PRV54986,1
10,PRV55193,1
13,PRV52283,1


(1082, 2)


### 6.3 Data Scaling
Now that we have our train and test set, we will proceed to scale our data using MinMax scaling.

In [291]:
# Initiate a Scaler
minmaxScaler = MinMaxScaler()

# Fit the scaler on train numerical data
numerical_cols = X_train.columns.difference(['ClaimID', 'Provider'])
minmaxScaler.fit(X_train[numerical_cols])

# Transform train and test numerical data
X_train[numerical_cols] = minmaxScaler.transform(X_train[numerical_cols])
X_test[numerical_cols] = minmaxScaler.transform(X_test[numerical_cols])

### 6.4 Creation of EntitySets
Next, we add our datasets into an entityset and map out their relationships. An EntitySet is a collection of dataframes and the relationship between each dataframe.

In [292]:
# Each dataset added into the entityset requires an index column that can uniquely identify every row.
# Hence we will first find a column to use as index in X_full
print(X_train.nunique().sort_values(
    ascending=False).head())  # We will use ClaimID

ClaimID                     447666
Provider                      4328
IPAnnualReimbursementAmt      2973
OPAnnualReimbursementAmt      2066
ClmDiagnosisCode_1_ce          948
dtype: int64


In [293]:
# Add data to entityset
entityset_train = ft.EntitySet(id="train")
entityset_train = entityset_train.add_dataframe(
    dataframe=X_train, dataframe_name="X_train", index="ClaimID",
)
entityset_train = entityset_train.add_dataframe(
    dataframe=y_train, dataframe_name="y_train", index="Provider"
)

entityset_test = ft.EntitySet(id="test")
entityset_test = entityset_test.add_dataframe(
    dataframe=X_test, dataframe_name="X_test", index="ClaimID",
)
entityset_test = entityset_test.add_dataframe(
    dataframe=y_test, dataframe_name="y_test", index="Provider"
)

display(entityset_train)
display(entityset_test)

Entityset: train
  DataFrames:
    X_train [Rows: 447666, Columns: 51]
    y_train [Rows: 4328, Columns: 2]
  Relationships:
    No relationships

Entityset: test
  DataFrames:
    X_test [Rows: 110545, Columns: 51]
    y_test [Rows: 1082, Columns: 2]
  Relationships:
    No relationships

In [294]:
# Next we outline the relationships between our dataframes
entityset_train = entityset_train.add_relationship(
    parent_dataframe_name='y_train',
    parent_column_name='Provider',
    child_dataframe_name='X_train',
    child_column_name='Provider'
)

entityset_test = entityset_test.add_relationship(
    parent_dataframe_name='y_test',
    parent_column_name='Provider',
    child_dataframe_name='X_test',
    child_column_name='Provider'
)

display(entityset_train)
display(entityset_test)

Entityset: train
  DataFrames:
    X_train [Rows: 447666, Columns: 51]
    y_train [Rows: 4328, Columns: 2]
  Relationships:
    X_train.Provider -> y_train.Provider

Entityset: test
  DataFrames:
    X_test [Rows: 110545, Columns: 51]
    y_test [Rows: 1082, Columns: 2]
  Relationships:
    X_test.Provider -> y_test.Provider

### 6.5 Deep Feature Synthesis
Now that we have our entitysets, we will carry out deep feature synthesis on the training and testing data.

In [295]:
# Now we run deep feature synthesis and create a feature matrix for each provider.
feature_matrix_providers_train, feature_defs_train = ft.dfs(
    entityset=entityset_train, target_dataframe_name="y_train")
feature_matrix_providers_test, feature_defs_test = ft.dfs(
    entityset=entityset_test, target_dataframe_name="y_test")

display(feature_matrix_providers_train.head())
print(feature_matrix_providers_train.shape)
display(feature_matrix_providers_test.head())
print(feature_matrix_providers_test.shape)

,PotentialFraud,COUNT(X_train),MAX(X_train.AdmitForDays),MAX(X_train.Age),MAX(X_train.AttendingPhysician_ce),MAX(X_train.ChronicCond_Alzheimer),MAX(X_train.ChronicCond_Cancer),MAX(X_train.ChronicCond_Depression),MAX(X_train.ChronicCond_Diabetes),MAX(X_train.ChronicCond_Heartfailure),MAX(X_train.ChronicCond_IschemicHeart),MAX(X_train.ChronicCond_KidneyDisease),MAX(X_train.ChronicCond_ObstrPulmonary),MAX(X_train.ChronicCond_Osteoporasis),MAX(X_train.ChronicCond_rheumatoidarthritis),MAX(X_train.ChronicCond_stroke),MAX(X_train.ClaimProcessedForDays),MAX(X_train.ClmAdmitDiagnosisCode_ce),MAX(X_train.ClmDiagnosisCode_10_ce),MAX(X_train.ClmDiagnosisCode_1_ce),MAX(X_train.ClmDiagnosisCode_2_ce),MAX(X_train.ClmDiagnosisCode_3_ce),MAX(X_train.ClmDiagnosisCode_4_ce),MAX(X_train.ClmDiagnosisCode_5_ce),MAX(X_train.ClmDiagnosisCode_6_ce),MAX(X_train.ClmDiagnosisCode_7_ce),MAX(X_train.ClmDiagnosisCode_8_ce),MAX(X_train.ClmDiagnosisCode_9_ce),MAX(X_train.ClmProcedureCode_1_ce),MAX(X_train.ClmProcedureCode_2_ce),MAX(X_train.ClmProcedureCode_3_ce),MAX(X_train.ClmProcedureCode_4_ce),MAX(X_train.ClmProcedureCode_5_ce),MAX(X_train.County),MAX(X_train.DeductibleAmtPaid),MAX(X_train.DiagnosisGroupCode_ce),MAX(X_train.Gender),MAX(X_train.IPAnnualDeductibleAmt),MAX(X_train.IPAnnualReimbursementAmt),MAX(X_train.InscClaimAmtReimbursed),MAX(X_train.NoOfMonths_PartACov),MAX(X_train.NoOfMonths_PartBCov),MAX(X_train.OPAnnualDeductibleAmt),MAX(X_train.OPAnnualReimbursementAmt),MAX(X_train.OperatingPhysician_ce),MAX(X_train.OtherPhysician_ce),MAX(X_train.Race),MAX(X_train.RenalDiseaseIndicator),MAX(X_train.State),MAX(X_train.isDead),MAX(X_train.is_inpatient),MEAN(X_train.AdmitForDays),MEAN(X_train.Age),MEAN(X_train.AttendingPhysician_ce),MEAN(X_train.ChronicCond_Alzheimer),MEAN(X_train.ChronicCond_Cancer),MEAN(X_train.ChronicCond_Depression),MEAN(X_train.ChronicCond_Diabetes),MEAN(X_train.ChronicCond_Heartfailure),MEAN(X_train.ChronicCond_IschemicHeart),MEAN(X_train.ChronicCond_KidneyDisease),MEAN(X_train.ChronicCond_ObstrPulmonary),MEAN(X_train.ChronicCond_Osteoporasis),MEAN(X_train.ChronicCond_rheumatoidarthritis),MEAN(X_train.ChronicCond_stroke),MEAN(X_train.ClaimProcessedForDays),MEAN(X_train.ClmAdmitDiagnosisCode_ce),MEAN(X_train.ClmDiagnosisCode_10_ce),MEAN(X_train.ClmDiagnosisCode_1_ce),MEAN(X_train.ClmDiagnosisCode_2_ce),MEAN(X_train.ClmDiagnosisCode_3_ce),MEAN(X_train.ClmDiagnosisCode_4_ce),MEAN(X_train.ClmDiagnosisCode_5_ce),MEAN(X_train.ClmDiagnosisCode_6_ce),MEAN(X_train.ClmDiagnosisCode_7_ce),MEAN(X_train.ClmDiagnosisCode_8_ce),MEAN(X_train.ClmDiagnosisCode_9_ce),MEAN(X_train.ClmProcedureCode_1_ce),MEAN(X_train.ClmProcedureCode_2_ce),MEAN(X_train.ClmProcedureCode_3_ce),MEAN(X_train.ClmProcedureCode_4_ce),MEAN(X_train.ClmProcedureCode_5_ce),MEAN(X_train.County),MEAN(X_train.DeductibleAmtPaid),MEAN(X_train.DiagnosisGroupCode_ce),MEAN(X_train.Gender),MEAN(X_train.IPAnnualDeductibleAmt),MEAN(X_train.IPAnnualReimbursementAmt),MEAN(X_train.InscClaimAmtReimbursed),MEAN(X_train.NoOfMonths_PartACov),MEAN(X_train.NoOfMonths_PartBCov),MEAN(X_train.OPAnnualDeductibleAmt),MEAN(X_train.OPAnnualReimbursementAmt),MEAN(X_train.OperatingPhysician_ce),MEAN(X_train.OtherPhysician_ce),MEAN(X_train.Race),MEAN(X_train.RenalDiseaseIndicator),MEAN(X_train.State),MEAN(X_train.isDead),MEAN(X_train.is_inpatient),MIN(X_train.AdmitForDays),MIN(X_train.Age),MIN(X_train.AttendingPhysician_ce),MIN(X_train.ChronicCond_Alzheimer),MIN(X_train.ChronicCond_Cancer),MIN(X_train.ChronicCond_Depression),MIN(X_train.ChronicCond_Diabetes),MIN(X_train.ChronicCond_Heartfailure),MIN(X_train.ChronicCond_IschemicHeart),MIN(X_train.ChronicCond_KidneyDisease),MIN(X_train.ChronicCond_ObstrPulmonary),MIN(X_train.ChronicCond_Osteoporasis),MIN(X_train.ChronicCond_rheumatoidarthritis),MIN(X_train.ChronicCond_stroke),MIN(X_train.ClaimProcessedForDays),MIN(X_train.ClmAdmitDiagnosisCode_ce),MIN(X_train.ClmDiagnosisCode_10_ce),MIN(X_train.ClmDiagnosisCode_1_ce),MIN(X_train.ClmDiagnosisCode_2_ce),MIN(X_tra

(4328, 296)


,PotentialFraud,COUNT(X_test),MAX(X_test.AdmitForDays),MAX(X_test.Age),MAX(X_test.AttendingPhysician_ce),MAX(X_test.ChronicCond_Alzheimer),MAX(X_test.ChronicCond_Cancer),MAX(X_test.ChronicCond_Depression),MAX(X_test.ChronicCond_Diabetes),MAX(X_test.ChronicCond_Heartfailure),MAX(X_test.ChronicCond_IschemicHeart),MAX(X_test.ChronicCond_KidneyDisease),MAX(X_test.ChronicCond_ObstrPulmonary),MAX(X_test.ChronicCond_Osteoporasis),MAX(X_test.ChronicCond_rheumatoidarthritis),MAX(X_test.ChronicCond_stroke),MAX(X_test.ClaimProcessedForDays),MAX(X_test.ClmAdmitDiagnosisCode_ce),MAX(X_test.ClmDiagnosisCode_10_ce),MAX(X_test.ClmDiagnosisCode_1_ce),MAX(X_test.ClmDiagnosisCode_2_ce),MAX(X_test.ClmDiagnosisCode_3_ce),MAX(X_test.ClmDiagnosisCode_4_ce),MAX(X_test.ClmDiagnosisCode_5_ce),MAX(X_test.ClmDiagnosisCode_6_ce),MAX(X_test.ClmDiagnosisCode_7_ce),MAX(X_test.ClmDiagnosisCode_8_ce),MAX(X_test.ClmDiagnosisCode_9_ce),MAX(X_test.ClmProcedureCode_1_ce),MAX(X_test.ClmProcedureCode_2_ce),MAX(X_test.ClmProcedureCode_3_ce),MAX(X_test.ClmProcedureCode_4_ce),MAX(X_test.ClmProcedureCode_5_ce),MAX(X_test.County),MAX(X_test.DeductibleAmtPaid),MAX(X_test.DiagnosisGroupCode_ce),MAX(X_test.Gender),MAX(X_test.IPAnnualDeductibleAmt),MAX(X_test.IPAnnualReimbursementAmt),MAX(X_test.InscClaimAmtReimbursed),MAX(X_test.NoOfMonths_PartACov),MAX(X_test.NoOfMonths_PartBCov),MAX(X_test.OPAnnualDeductibleAmt),MAX(X_test.OPAnnualReimbursementAmt),MAX(X_test.OperatingPhysician_ce),MAX(X_test.OtherPhysician_ce),MAX(X_test.Race),MAX(X_test.RenalDiseaseIndicator),MAX(X_test.State),MAX(X_test.isDead),MAX(X_test.is_inpatient),MEAN(X_test.AdmitForDays),MEAN(X_test.Age),MEAN(X_test.AttendingPhysician_ce),MEAN(X_test.ChronicCond_Alzheimer),MEAN(X_test.ChronicCond_Cancer),MEAN(X_test.ChronicCond_Depression),MEAN(X_test.ChronicCond_Diabetes),MEAN(X_test.ChronicCond_Heartfailure),MEAN(X_test.ChronicCond_IschemicHeart),MEAN(X_test.ChronicCond_KidneyDisease),MEAN(X_test.ChronicCond_ObstrPulmonary),MEAN(X_test.ChronicCond_Osteoporasis),MEAN(X_test.ChronicCond_rheumatoidarthritis),MEAN(X_test.ChronicCond_stroke),MEAN(X_test.ClaimProcessedForDays),MEAN(X_test.ClmAdmitDiagnosisCode_ce),MEAN(X_test.ClmDiagnosisCode_10_ce),MEAN(X_test.ClmDiagnosisCode_1_ce),MEAN(X_test.ClmDiagnosisCode_2_ce),MEAN(X_test.ClmDiagnosisCode_3_ce),MEAN(X_test.ClmDiagnosisCode_4_ce),MEAN(X_test.ClmDiagnosisCode_5_ce),MEAN(X_test.ClmDiagnosisCode_6_ce),MEAN(X_test.ClmDiagnosisCode_7_ce),MEAN(X_test.ClmDiagnosisCode_8_ce),MEAN(X_test.ClmDiagnosisCode_9_ce),MEAN(X_test.ClmProcedureCode_1_ce),MEAN(X_test.ClmProcedureCode_2_ce),MEAN(X_test.ClmProcedureCode_3_ce),MEAN(X_test.ClmProcedureCode_4_ce),MEAN(X_test.ClmProcedureCode_5_ce),MEAN(X_test.County),MEAN(X_test.DeductibleAmtPaid),MEAN(X_test.DiagnosisGroupCode_ce),MEAN(X_test.Gender),MEAN(X_test.IPAnnualDeductibleAmt),MEAN(X_test.IPAnnualReimbursementAmt),MEAN(X_test.InscClaimAmtReimbursed),MEAN(X_test.NoOfMonths_PartACov),MEAN(X_test.NoOfMonths_PartBCov),MEAN(X_test.OPAnnualDeductibleAmt),MEAN(X_test.OPAnnualReimbursementAmt),MEAN(X_test.OperatingPhysician_ce),MEAN(X_test.OtherPhysician_ce),MEAN(X_test.Race),MEAN(X_test.RenalDiseaseIndicator),MEAN(X_test.State),MEAN(X_test.isDead),MEAN(X_test.is_inpatient),MIN(X_test.AdmitForDays),MIN(X_test.Age),MIN(X_test.AttendingPhysician_ce),MIN(X_test.ChronicCond_Alzheimer),MIN(X_test.ChronicCond_Cancer),MIN(X_test.ChronicCond_Depression),MIN(X_test.ChronicCond_Diabetes),MIN(X_test.ChronicCond_Heartfailure),MIN(X_test.ChronicCond_IschemicHeart),MIN(X_test.ChronicCond_KidneyDisease),MIN(X_test.ChronicCond_ObstrPulmonary),MIN(X_test.ChronicCond_Osteoporasis),MIN(X_test.ChronicCond_rheumatoidarthritis),MIN(X_test.ChronicCond_stroke),MIN(X_test.ClaimProcessedForDays),MIN(X_test.ClmAdmitDiagnosisCode_ce),MIN(X_test.ClmDiagnosisCode_10_ce),MIN(X_test.ClmDiagnosisCode_1_ce),MIN(X_test.ClmDiagnosisCode_2_ce),MIN(X_test.ClmDiagnosisCode_3_ce),MIN(X_test.ClmDiagnosisCode_4_ce),MIN(X_test.ClmDiagnosisCode_5_ce),MIN(X_test.ClmDiagnosisCo

(1082, 296)


### Dimensionality Reduction -- Principal Component Analysis

KIV: can include gridsearch/ kfold validation here to determine best number of PCA

In [296]:
from sklearn.decomposition import PCA

# Perform PCA with 3 components on the standard scaled dataset
pca = PCA(n_components=10, random_state=0)
pca.fit(X_train_standard_scaled[features_to_scale])

# Create a DataFrame to store the PCA results
PCA_ds = pd.DataFrame(pca.transform(X_train_standard_scaled[features_to_scale]), 
                      columns=["col1", "col2", "col3","col4","col5",'col6','col7','col8','col9','col10'])

# Display summary statistics of the PCA results
PCA_summary = PCA_ds.describe().T
print(PCA_summary)

NameError: name 'X_train_standard_scaled' is not defined

In [ ]:
PC_values = np.arange(pca.n_components_) + 1
plt.plot(PC_values, pca.explained_variance_ratio_, 'o-', linewidth=2, color='blue')
plt.title('Scree Plot for Standard Scaler')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.show()

Ideal PC = 2 for standard scaler

In [ ]:
# Perform PCA with 3 components on the minmax scaled dataset
pca_minmax = PCA(n_components=10, random_state=0)
pca_minmax.fit(X_train_minmax_scaled[features_to_scale])

# Create a DataFrame to store the PCA results
PCA_ds = pd.DataFrame(pca_minmax.transform(X_train_minmax_scaled[features_to_scale]), 
                      columns=["col1", "col2", "col3","col4","col5",'col6','col7','col8','col9','col10'])

# Display summary statistics of the PCA results
PCA_summary = PCA_ds.describe().T
print(PCA_summary)

In [ ]:
PC_values = np.arange(pca_minmax.n_components_) + 1
plt.plot(PC_values, pca_minmax.explained_variance_ratio_, 'o-', linewidth=2, color='blue')
plt.title('Scree Plot for Min Max Scaler')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.show()

Ideal PC = 4